# Analysis of Bogotá's household data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

%matplotlib inline

## Catastro

In [2]:
df_catastro = pd.read_csv('Info_catastro.csv', low_memory = False)
df_predios = pd.read_table('predios.txt', on_bad_lines='warn', delimiter=';')

C:\Users\mdomi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df_catastro['Codigo'] = df_catastro['CONCODIGO'].apply(lambda df: df[:12])
df_predios['Codigo'] = df_predios.apply(lambda df:  f'{df.PreCBarrio:06}' + f'{df.PreCManz:03}' + f'{df.PreCPredio:03}', axis =1)
df_predios = df_predios[(df_predios['PreVetustz']>1800)&(df_predios['PreCUso'] == 1)].iloc[:,[21, -1]]
df_predios['AGE_RANK'] = df_predios['PreVetustz'].apply(lambda df: 1 if (2022 - df)<10
                                                        else 2 if ((2022-df)<20) & ((2022-df)>10) 
                                                        else 3 if ((2022-df)<30) & ((2022-df)>20) 
                                                        else 4 if ((2022-df)<40) & ((2022-df)>30) 
                                                        else 5 if ((2022-df)<50) & ((2022-df)>40)
                                                        else 6 if ((2022-df)<60) & ((2022-df)>50)
                                                        else 7 if ((2022-df)<70) & ((2022-df)>60)
                                                        else 8 if ((2022-df)<80) & ((2022-df)>70)
                                                        else 9
                                                       )
df_predios = df_predios.iloc[:,-2:]
df_catastro = df_catastro.set_index('Codigo').iloc[:,[2, 4]]

In [4]:
houses = df_predios.merge(df_catastro, left_on = 'Codigo', right_on = 'Codigo')
houses = houses.groupby('Codigo').max().reset_index()
houses['BASEMENT'] = houses.apply(lambda df : 1 if df.CONNSOTANO > 0 else 0, axis = 1)
houses['COD'] = houses.Codigo.apply(lambda df : 'A' + df)
houses = houses.iloc[:,[5,4,1]]
houses

,COD,BASEMENT,AGE_RANK
0,A001101002001,0,5
1,A001101002002,0,9
2,A001101002003,0,5
3,A001101002004,0,3
4,A001101002005,0,5
...,...,...,...
725936,A209106005023,0,3
725937,A209106005024,0,4
725938,A209106005025,0,4
725939,A209106006001,0,4


In [5]:
houses.to_csv('construction_vars_houses_Bogotá.csv')